In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [5]:
data = pd.read_csv('/content/Stock_data.csv')

In [6]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))


In [7]:
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

In [8]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

In [9]:
seq_length = 10
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

In [10]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

In [11]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
51/51 [==============================] - 7s 17ms/step - loss: 0.0213
Epoch 2/10
51/51 [==============================] - 1s 15ms/step - loss: 9.2990e-04
Epoch 3/10
51/51 [==============================] - 1s 15ms/step - loss: 9.1936e-04
Epoch 4/10
51/51 [==============================] - 1s 15ms/step - loss: 8.5420e-04
Epoch 5/10
51/51 [==============================] - 1s 16ms/step - loss: 8.4861e-04
Epoch 6/10
51/51 [==============================] - 1s 18ms/step - loss: 8.6547e-04
Epoch 7/10
51/51 [==============================] - 1s 26ms/step - loss: 8.2344e-04
Epoch 8/10
51/51 [==============================] - 1s 26ms/step - loss: 8.7056e-04
Epoch 9/10
51/51 [==============================] - 2s 29ms/step - loss: 8.2019e-04
Epoch 10/10
51/51 [==============================] - 1s 25ms/step - loss: 8.2426e-04


In [12]:
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

13/13 [==============================] - 0s 6ms/step


In [13]:
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)


In [14]:
train_rmse = np.sqrt(np.mean((train_predictions - scaler.inverse_transform(y_train)) ** 2))
test_rmse = np.sqrt(np.mean((test_predictions - scaler.inverse_transform(y_test)) ** 2))
print('Train RMSE:', train_rmse)
print('Test RMSE:', test_rmse)

Train RMSE: 6.723485195195259
Test RMSE: 5.388006013328859
